In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import re
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import ast
from datetime import datetime
from itertools import chain

# Load Data 

In [3]:
# load league data
league_data = pd.read_csv('comb_leagues_format_avg_stats.csv')
# extract tottenham data
tot_data = league_data.query('HomeTeam == "Tottenham" or AwayTeam == "Tottenham"')

# Remove Redundant Cols 

In [5]:
# remove irrelevant columns
tot_data = tot_data[['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG','HS','AS','HST','AST',
                     'HF','AF','HC','AC','HY','AY','HR','AR','FTR']]

# Create Points Columns

In [6]:
# create points columns
tot_data['HomePoints'] = 0
tot_data['AwayPoints'] = 0
# query home win
hw_idx = tot_data.query('FTHG > FTAG').index
tot_data.loc[hw_idx, 'HomePoints'] = 3
tot_data.loc[hw_idx, 'AwayPoints'] = 0
# query away win
aw_idx = tot_data.query('FTAG > FTHG').index
tot_data.loc[aw_idx , 'HomePoints'] = 0
tot_data.loc[aw_idx, 'AwayPoints'] = 3
# query draw
d_idx = tot_data.query('FTHG == FTAG').index
tot_data.loc[d_idx , 'HomePoints'] = 1
tot_data.loc[d_idx, 'AwayPoints'] = 1

<ipython-input-6-5446b4b3d921>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['HomePoints'] = 0
<ipython-input-6-5446b4b3d921>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['AwayPoints'] = 0
C:\Users\jakee\anaconda3\envs\football_team_estimation\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

# Convert Data Types

In [8]:
# convert float dtypes to int
tot_data['HS'] = tot_data['HS'].astype('int64')
tot_data['AS'] = tot_data['AS'].astype('int64')
tot_data['HST'] = tot_data['HST'].astype('int64')
tot_data['AST'] = tot_data['AST'].astype('int64')
tot_data['HF'] = tot_data['HF'].astype('int64')
tot_data['AF'] = tot_data['AF'].astype('int64')
tot_data['HC'] = tot_data['HC'].astype('int64')
tot_data['AC'] = tot_data['AC'].astype('int64')
tot_data['HY'] = tot_data['HY'].astype('int64')
tot_data['AY'] = tot_data['AY'].astype('int64')
tot_data['HR'] = tot_data['HR'].astype('int64')
tot_data['AR'] = tot_data['AR'].astype('int64')

<ipython-input-8-07b8c71fa9b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['HS'] = tot_data['HS'].astype('int64')
<ipython-input-8-07b8c71fa9b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['AS'] = tot_data['AS'].astype('int64')
<ipython-input-8-07b8c71fa9b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [9]:
tot_data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,...,AF,HC,AC,HY,AY,HR,AR,FTR,HomePoints,AwayPoints
8,E0,19/08/00,Tottenham,Ipswich,3,1,20,15,6,5,...,13,3,4,0,0,0,0,H,3,0
13,E0,22/08/00,Middlesbrough,Tottenham,1,1,12,11,6,4,...,18,5,5,2,1,0,0,D,1,1
25,E0,26/08/00,Newcastle,Tottenham,2,0,15,10,6,2,...,10,4,2,0,0,0,0,H,3,0
32,E0,05/09/00,Tottenham,Everton,3,2,11,11,7,6,...,10,7,5,1,2,0,0,H,3,0
48,E0,11/09/00,Tottenham,West Ham,1,0,15,11,7,5,...,13,11,3,1,2,0,0,H,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42831,E0,01/05/2022,Tottenham,Leicester,3,1,13,6,7,2,...,8,7,3,2,3,0,0,H,3,0
42839,E0,07/05/2022,Liverpool,Tottenham,1,1,22,8,3,3,...,8,11,5,3,2,0,0,D,1,1
42849,E0,12/05/2022,Tottenham,Arsenal,3,0,16,8,6,4,...,15,2,2,1,3,0,1,H,3,0
42850,E0,15/05/2022,Tottenham,Burnley,1,0,21,8,8,1,...,8,8,2,3,2,0,0,H,3,0


In [12]:
# convert date to datetime
tot_data['Date'] = pd.to_datetime(tot_data['Date'], dayfirst=True)

<ipython-input-12-3b660627cbcd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['Date'] = pd.to_datetime(tot_data['Date'], dayfirst=True)


# Create Season Column

In [16]:
## create season data
# add season column
tot_data['Season'] = np.nan
# add season data
for i in range(22):
    # subset season
    if i < 9:
        season_idx = tot_data.query(f'200{i}0801 < Date < 200{i+1}0801').index
        tot_data.loc[season_idx, 'Season'] = f'0{i}/0{i+1}'
    elif i == 9:
        season_idx = tot_data.query(f'200{i}0801 < Date < 20{i+1}0801').index
        tot_data.loc[season_idx, 'Season'] = f'0{i}/{i+1}'
    else:
        season_idx = tot_data.query(f'20{i}0801 < Date < 20{i+1}0801').index
        tot_data.loc[season_idx, 'Season'] = f'{i}/{i+1}'

<ipython-input-16-db8f713009f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tot_data['Season'] = np.nan
C:\Users\jakee\anaconda3\envs\football_team_estimation\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
tot_data

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,AST,...,HC,AC,HY,AY,HR,AR,FTR,HomePoints,AwayPoints,Season
8,E0,2000-08-19,Tottenham,Ipswich,3,1,20,15,6,5,...,3,4,0,0,0,0,H,3,0,00/01
13,E0,2000-08-22,Middlesbrough,Tottenham,1,1,12,11,6,4,...,5,5,2,1,0,0,D,1,1,00/01
25,E0,2000-08-26,Newcastle,Tottenham,2,0,15,10,6,2,...,4,2,0,0,0,0,H,3,0,00/01
32,E0,2000-09-05,Tottenham,Everton,3,2,11,11,7,6,...,7,5,1,2,0,0,H,3,0,00/01
48,E0,2000-09-11,Tottenham,West Ham,1,0,15,11,7,5,...,11,3,1,2,0,0,H,3,0,00/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42831,E0,2022-05-01,Tottenham,Leicester,3,1,13,6,7,2,...,7,3,2,3,0,0,H,3,0,21/22
42839,E0,2022-05-07,Liverpool,Tottenham,1,1,22,8,3,3,...,11,5,3,2,0,0,D,1,1,21/22
42849,E0,2022-05-12,Tottenham,Arsenal,3,0,16,8,6,4,...,2,2,1,3,0,1,H,3,0,21/22
42850,E0,2022-05-15,Tottenham,Burnley,1,0,21,8,8,1,...,8,2,3,2,0,0,H,3,0,21/22


In [18]:
# save data
tot_data.to_csv('tottenham_prem_data_new.csv', index=False, encoding='utf-8')

# Split Data (Home/Away)

In [19]:
# create home match data
tot_home_data = tot_data.query('HomeTeam == "Tottenham"')
tot_home_data.to_csv('tottenham_home_prem_data_new.csv', index=False, encoding='utf-8')

In [20]:
# create away match data
tot_away_data = tot_data.query('AwayTeam == "Tottenham"')
tot_away_data.to_csv('tottenham_away_prem_data_new.csv', index=False, encoding='utf-8')

# Goal Scorer Scrape

In [15]:
## goal scorer and minutes scored scrape script
# season & club url codes
season_code = {
    '2000/01': 9,
    '2001/02': 10,
    '2002/03': 11,
    '2003/04': 12,
    '2004/05': 13,
    '2005/06': 14,
    '2006/07': 15,
    '2007/08': 16,
    '2008/09': 17,
    '2009/10': 18,
    '2010/11': 19,
    '2011/12': 20,
    '2012/13': 21,
    '2013/14': 22,
    '2014/15': 27,
    '2015/16': 42,
    '2016/17': 54,
    '2017/18': 79,
    '2018/19': 210,
    '2019/20': 274,
    '2020/21': 363,
    '2021/22': 418
}
# tottenham code
spurs = 21

# lists to store scraped data
dates = []
seasons = []
home_team = []
away_team = []
tot_scorers = []

## main code
for season, code in season_code.items():
    #### get season game id's
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options, executable_path=r'C:/Users/jakee/Documents/chrome_driver/bin/chromedriver.exe')
    wait = WebDriverWait(driver, 40)
    driver.get(f'https://www.premierleague.com/results?co=1&se={code}&cl={spurs}') # specific season 2000/01


    time.sleep(5) ## pause 5 secs
    # grab fixture dates
    date = driver.find_elements(By.CSS_SELECTOR, "time[class='date long']")
    fixture_dates = []
    for d in date:
        d_html = driver.execute_script("return arguments[0].innerHTML;", d)
        d_re = re.search('\>(.*?)\<', d_html)[0]
        fixture_dates.append(d_re[1:-1])

    # grab fixtures in javascript with selenium
    fixtures = driver.find_element(By.CSS_SELECTOR, "section[class='fixtures']") # grab fixtures
    fixtures_html = driver.execute_script("return arguments[0].innerHTML;", fixtures) # turn to html
    # parse html with beautiful soup
    fixtures_soup = BeautifulSoup(fixtures_html, 'html.parser')
    matches = fixtures_soup.find_all('li', class_='matchFixtureContainer') # grab each match fixture
    # extract data from match fixture
    season_match_list = []
    for i,_ in enumerate(matches):
        match_dict = {
            'Date': fixture_dates[i],
            'HomeTeam': str(matches[i].get('data-home')),
            'AwayTeam': str(matches[i].get('data-away')),
            'MatchId': int(matches[i].get('data-comp-match-item'))
        }
        season_match_list.append(match_dict)
    driver.quit()
    season_match_list = [x for x in reversed(season_match_list)] # reverse list for correct fixture order
    #season_match_list

    for fixture in season_match_list:
        dates.append(fixture.get('Date'))
        seasons.append(season) 
        home_team.append(fixture.get('HomeTeam'))
        away_team.append(fixture.get('AwayTeam'))
        time.sleep(random.uniform(4,9)) ## pause for x secs
        # scrape web page
        #match_fixture = season_match_list[1]
        match_id = fixture.get('MatchId') ## change when full run
        match_url = f'https://www.premierleague.com/match/{match_id}' # match web url
        match_page = requests.get(match_url)
        match_soup = BeautifulSoup(match_page.content, 'html.parser')

        # scrape goal scorers and mins scored
        # get spurs data for home or away games
        if fixture.get('HomeTeam') == 'Spurs':
            stadium = 'home'
        else:
            stadium = 'away'
        match_result = match_soup.find('div', {'class': 'matchEvents matchEventsContainer'}) # gets match events
        spurs_events = match_result.find('div', {'class':f'{stadium}'}) # find home or away event
        find_mins = spurs_events.find_all(string=True)

        # get scorers
        p = [re.search("[A-Za-z- àèìòùáéíóúýâêîôûãñõäëïöüÿæçøå]+", x) for x in find_mins]
        p = [x[0] for x in p if x is not None] # remove None
        #p = [x for x in p if x != ' ' and x != 'Goal' and x != 'Own Goal'] # remove ' ' & 'Goal'
        p = [x for x in p if x != ' '] # remove empty string ' '
        event_group = lambda l, n: [tuple(l[i:i+n]) for i in range(0, len(l), n)] # groups player name with event
        p = event_group(p,2) # groups player name with event
        # remove event if not a goal
        p = [x[0] for x in p if 'Second Yellow Card' not in x[1] and 'Yellow Card' not in x[1] and 'Red Card' not in x[1]]

        # get minutes scored
        gm = [re.search("[0-9\s\+,'(a-z)]+", x)[0] for x in find_mins]
        gm = [x for x in gm if x != '\n'] # remove '\n'
        gm = [x for x in gm if x[0] == ' '] # goals have ' ' at start
        gm = [x.split('\n') for x in gm] # split '\n        '
        gm = [x[0] for x in gm] # remove whitespace

        # move own goal next to scorer... ### convert mins from strings to ints
        mins_scored = []
        for i,e in enumerate(gm):
            if '(og)' in e: # if own goal label scorer with (own goal)
                p[i] = p[i] + ' (own goal)'
                e = e.replace('(og)', '') # remove own goal from mins
                e = e.replace("'", "") # remove apostrophe
                e = e.replace(" ", "") # remove space
                e = e.split(',') # split on ',' .. seperates if more than 2 goals scored 
                # turn string mins to int mins, sum injury time .. e.g. '90+3'
                e = [int(x) if '+' not in x else sum(map(int, x.split('+'))) for x in e] 
                mins_scored.append(e)
            else: # if not own goal
                e = re.sub('[(a-z)]', '', e) # remove any other events that are not mins
                e = e.replace("'", '') # remove apostrophe
                e = e.replace(" ", "") # remove space
                e = e.split(',') # split on ',' .. seperates if more than 2 goals scored 
                # turn string mins to int mins, sum injury time .. e.g. '90+3'
                e = [int(x) if '+' not in x else sum(map(int, x.split('+'))) for x in e] 
                mins_scored.append(e)

        tot_scorers.append(dict(zip(p, mins_scored)))
    print(f'Season {season} scraped.')
    
scorer_dict = {
    'Date': dates,
    'Season': seasons,
    'HomeTeam': home_team,
    'AwayTeam': away_team,
    'Scorers': tot_scorers
}
scorer_df = pd.DataFrame(scorer_dict)

# save scorer dataframe
scorer_df.to_csv('tottenham_scorer_df.csv', index=False, encoding='utf-8')

<ipython-input-15-73c3f92d65f0>:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r'C:/Users/jakee/Documents/chrome_driver/bin/chromedriver.exe')


Season 2000/01 scraped.
Season 2001/02 scraped.
Season 2002/03 scraped.
Season 2003/04 scraped.
Season 2004/05 scraped.
Season 2005/06 scraped.
Season 2006/07 scraped.
Season 2007/08 scraped.
Season 2008/09 scraped.
Season 2009/10 scraped.
Season 2010/11 scraped.
Season 2011/12 scraped.
Season 2012/13 scraped.
Season 2013/14 scraped.
Season 2014/15 scraped.
Season 2015/16 scraped.
Season 2016/17 scraped.
Season 2017/18 scraped.
Season 2018/19 scraped.
Season 2019/20 scraped.
Season 2020/21 scraped.
Season 2021/22 scraped.


In [2]:
pd.read_csv('tottenham_scorer_df.csv')

,Date,Season,HomeTeam,AwayTeam,Scorers
0,Saturday 19 August 2000,2000/01,Spurs,Ipswich,"{'Darren Anderton': [30], 'Stephen Carr': [31]..."
1,Tuesday 22 August 2000,2000/01,Middlesbrough,Spurs,{'Oyvind Leonhardsen': [41]}
2,Saturday 26 August 2000,2000/01,Newcastle,Spurs,{}
3,Tuesday 5 September 2000,2000/01,Spurs,Everton,"{'Serhiy Rebrov': [45, 61], 'Les Ferdinand': [..."
4,Monday 11 September 2000,2000/01,Spurs,West Ham,{'Sol Campbell': [67]}
...,...,...,...,...,...
831,Sunday 1 May 2022,2021/22,Spurs,Leicester,"{'Harry Kane': [22], 'Son Heung-Min': [60, 79]}"
832,Saturday 7 May 2022,2021/22,Liverpool,Spurs,{'Son Heung-Min': [56]}
833,Thursday 12 May 2022,2021/22,Spurs,Arsenal,"{'Harry Kane': [22, 37], 'Son Heung-Min': [47]}"
834,Sunday 15 May 2022,2021/22,Spurs,Burnley,{'Harry Kane': [53]}


# Create NumGoals & MinsScored Cols

In [2]:
gs_data = pd.read_csv('tottenham_scorer_df.csv')

In [83]:
# create dataframe with num goals per player
index = []
date = []
season = []
home_team = []
away_team = []
player = []
num_goals = []

for i in range(len(gs_data)):
    fxt = gs_data.iloc[i,:] # take row from df
    s = ast.literal_eval(fxt['Scorers']) # convert string dict to dict
    if len(s) == 0: # if no goals scored
        index.append(i)
        # convert date to datetime
        d = datetime.strptime(fxt['Date'], '%A %d %B %Y')
        date.append(pd.to_datetime(f'{d.day}/{d.month}/{d.year}', dayfirst=True))
        season.append(fxt['Season'])
        home_team.append(fxt['HomeTeam'])
        away_team.append(fxt['AwayTeam'])
        player.append('None')
        num_goals.append(0)
    else: # if goals scores
        for k,v in s.items(): # loop through dict
            if '(own goal)' in k:
                continue
            else:
                index.append(i)
                # convert date to datetime
                d = datetime.strptime(fxt['Date'], '%A %d %B %Y')
                date.append(pd.to_datetime(f'{d.day}/{d.month}/{d.year}', dayfirst=True))
                season.append(fxt['Season'])
                home_team.append(fxt['HomeTeam'])
                away_team.append(fxt['AwayTeam'])
                player.append(k)
                num_goals.append(len(v))

goals_df = pd.DataFrame(
    {
        'GameIndex': index,
        'Date': date,
        'HomeTeam': home_team,
        'AwayTeam': away_team,
        'Player': player,
        'NumGoals': num_goals
    }
)
goals_df.set_index('GameIndex')
# save dataframe
goals_df.to_csv('spurs_goals_scored_data.csv', index=False, encoding='utf-8')

In [53]:
## match team names according to original data
tot_data = pd.read_csv('tottenham_prem_data.csv')
goals_data = pd.read_csv('spurs_goals_scored_data.csv')

names_to_change = ['Man Utd', 'Sheffield Utd', 'Spurs']
names_change_to = ['Man United', 'Sheffield United', 'Tottenham']
for i in range(len(names_to_change)):
    # home team names
    hteam_idx = goals_data.query(f"HomeTeam == '{names_to_change[i]}'").index
    goals_data.loc[hteam_idx, 'HomeTeam'] = names_change_to[i]
    # away team names
    ateam_idx = goals_data.query(f"AwayTeam == '{names_to_change[i]}'").index
    goals_data.loc[ateam_idx, 'AwayTeam'] = names_change_to[i]

# save dataframe
goals_data.to_csv('spurs_goals_scored_data.csv', index=False, encoding='utf-8')

In [58]:
## split goals data into home and away data
goals_data = pd.read_csv('spurs_goals_scored_data.csv')
home_goals_data = goals_data.query("HomeTeam=='Tottenham'")
away_goals_data = goals_data.query("AwayTeam=='Tottenham'")
# save dataframes
home_goals_data.to_csv('home_spurs_goals_data.csv', index=False, encoding='utf-8')
away_goals_data.to_csv('away_spurs_goals_data.csv', index=False, encoding='utf-8')

In [100]:
# create dataframe with mins scored per player
index = []
date = []
season = []
home_team = []
away_team = []
player = []
mins_scored = []

for i in range(len(gs_data)):
    fxt = gs_data.iloc[i,:] # take row from df
    s = ast.literal_eval(fxt['Scorers']) # convert string dict to dict
    if len(s) == 0: # if no goals scored
        index.append(i)
        # convert date to datetime
        d = datetime.strptime(fxt['Date'], '%A %d %B %Y')
        date.append(pd.to_datetime(f'{d.day}/{d.month}/{d.year}', dayfirst=True))
        season.append(fxt['Season'])
        home_team.append(fxt['HomeTeam'])
        away_team.append(fxt['AwayTeam'])
        player.append('None')
        mins_scored.append([-999]) ## negative number to stand for no mins, to keep data type as int
    else: # if goals scores
        for k,v in s.items(): # loop through dict
            if '(own goal)' in k:
                continue
            else:
                index.append(i)
                # convert date to datetime
                d = datetime.strptime(fxt['Date'], '%A %d %B %Y')
                date.append(pd.to_datetime(f'{d.day}/{d.month}/{d.year}', dayfirst=True))
                season.append(fxt['Season'])
                home_team.append(fxt['HomeTeam'])
                away_team.append(fxt['AwayTeam'])
                player.append(k)
                mins_scored.append(v)

mins_df = pd.DataFrame(
    {
        'GameIndex': index,
        'Date': date,
        'HomeTeam': home_team,
        'AwayTeam': away_team,
        'Player': player,
        'MinsScored': mins_scored
    }
)
mins_df.set_index('GameIndex')
# save dataframe
mins_df.to_csv('spurs_mins_scored_data.csv', index=False, encoding='utf-8')